In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:
rows = []

for page_num in range(1, 5):
    url = f"http://www.metrolyrics.com/rem-alpage-{page_num}"
    # print("Now scraping", url)
    
    response = requests.get(url)
    doc = BeautifulSoup(response.text)
    
    songs = doc.find('table').find_all('tr')
    for song in songs[1:]:
        row = {}
        row['Song title'] = song.find(class_="title").text.strip()
        row['URL'] = song.find('a')['href']
        row['Popularity'] = song.find_all('td')[3].find_all('span')[1].get('style')
        row['Year'] = song.find_all('td')[2].text.strip()
        rows.append(row)

df = pd.DataFrame(rows)

In [3]:
df['Popularity'] = df['Popularity'].str.extract("width:(.*)%;", expand=False)

In [4]:
df = df[['Song title', 'URL', 'Popularity', 'Year']]
df.head()

,Song title,URL,Popularity,Year
0,#9 Dream Lyrics,http://www.metrolyrics.com/9-dream-lyrics-rem....,6,2007
1,"1,000,000 Lyrics",http://www.metrolyrics.com/1000000-lyrics-rem....,6,2006
2,11 Lyrics,http://www.metrolyrics.com/11-lyrics-rem.html,6,2007
3,11th Untitled Song Lyrics,http://www.metrolyrics.com/11th-untitled-song-...,6,2006
4,7 Chinese Brothers Lyrics,http://www.metrolyrics.com/7-chinese-brothers-...,6,2007


In [5]:
df.to_csv("song-list.csv", index=False)

# Metrolyrics, Part 2: Scrape the lyrics pages

In [6]:
import pandas as pd

In [7]:
df = pd.read_csv("song-list.csv")

In [8]:
def scrape_page(row):
    # print("Scraping", row['URL'])
    url = row['URL']
    
    response = requests.get(url)
    doc = BeautifulSoup(response.text)

    page = {}
    lyrics = doc.find_all(class_='verse')
    page['Lyrics'] = '\n'.join([lyric.text.strip() for lyric in lyrics])

    return pd.Series(page)

In [9]:
scraped_df = df.apply(scrape_page, axis=1)

In [10]:
df2 = df.merge(scraped_df, left_index=True, right_index=True)
df2.head()

,Song title,URL,Popularity,Year,Lyrics
0,#9 Dream Lyrics,http://www.metrolyrics.com/9-dream-lyrics-rem....,6.0,2007,"It was so long ago, was it in a dream?\nWas it..."
1,"1,000,000 Lyrics",http://www.metrolyrics.com/1000000-lyrics-rem....,6.0,2006,Secluded in a marker stone not only deadlier\n...
2,11 Lyrics,http://www.metrolyrics.com/11-lyrics-rem.html,6.0,2007,
3,11th Untitled Song Lyrics,http://www.metrolyrics.com/11th-untitled-song-...,6.0,2006,This world is big and so-awake\nI stayed up la...
4,7 Chinese Brothers Lyrics,http://www.metrolyrics.com/7-chinese-brothers-...,6.0,2007,"This mellow, sweet, short-haired boy\nWoman of..."


In [11]:
df2.to_csv("song-lyrics.csv", index=False)